In [2]:
import pandas as pd
import numpy as np  

Read the file

In [3]:
file = pd.read_excel('shortcourses2566.xlsx')

Create Series of users

In [4]:
s_name = file.loc[:, 'ชื่อ-นามสกุล (อังกฤษ)']
Users = pd.Series(s_name, name='Users')
Users

0             PORPHAING JANTIP
1         THANAKORN DARASRISAK
2             KARNJANA EAMTANG
3             KARNJANA EAMTANG
4       THITIMUN VORATHONGCHAI
                 ...          
6122         KITTINON SANTASUP
6123      WITTAWAT SERMSRIPONG
6124               NON NAKKARA
6125         PHATTHARAPHON LIN
6126      THANAPAT SEEMAKAJOHN
Name: Users, Length: 6127, dtype: object

Create Series of emails

In [5]:
s_email = file.loc[:, 'อีเมล'].fillna("")
Emails = pd.Series(s_email ,name='Emails')
Emails


0            tanghaoren17@gamil.com
1               tuakung21@gmail.com
2            karnjana.aon@gmail.com
3            karnjana.aon@gmail.com
4          thitimun.rama1@gmail.com
                   ...             
6122           kittinon_s@cmu.ac.th
6123        wittawat_serm@cmu.ac.th
6124          non_nakkara@cmu.ac.th
6125    phattharaphon_lin@cmu.ac.th
6126         thanapat_see@cmu.ac.th
Name: Emails, Length: 6127, dtype: object

Provide a score to each user based on their email domain

In [6]:
email_score = []
for data in Emails:
    if data != '':
        if data.split('@')[1] == 'cmu.ac.th':
            email_score.append(2)
        else:
            email_score.append(1)
    else:
        email_score.append(0)
email_score = pd.Series(email_score, name='Score Emails')
email_score

0       1
1       1
2       1
3       1
4       1
       ..
6122    2
6123    2
6124    2
6125    2
6126    2
Name: Score Emails, Length: 6127, dtype: int64

Create function to calculate age-education score

In [7]:
def getAgeEducationScore(age, limit_age):
    if age <= limit_age:
        score = 1
    else:
        score = 3
    return score

Create Series of Age-Education

In [8]:
ages = file.loc[:, 'อายุ']
educations = file.loc[:, 'วุฒิการศึกษา']
age_education_scores = []
# check output length of age_education_scores until it has 6127
for i,x in enumerate(educations):
    if x == 'ประถมศึกษา':
        age_education_scores.append(getAgeEducationScore(ages[i], 16))
    if x == 'มัธยมศึกษาตอนต้น':
        age_education_scores.append(getAgeEducationScore(ages[i], 19))
    if x == 'มัธยมศึกษาตอนปลาย' or x == 'High school':
        age_education_scores.append(getAgeEducationScore(ages[i], 22))
    if x == 'ปริญญาตรี':
        age_education_scores.append(getAgeEducationScore(ages[i], 26))
print(len(age_education_scores))

5102


Create Series of status

In [13]:
status = file.loc[:, 'สถานะ'].fillna("")
status = pd.Series(status ,name='status')

Provide a score to each user based on their purchase status

In [16]:
status_score = []
for x in status:
    if x == 'ชำระเงิน':
        status_score.append(8)
    if x == 'ไม่ผ่านการอนุมัติ':
        status_score.append(7)
    if x == 'ค้างชำระ':
        status_score.append(5)

Create Series of address

In [20]:
address = file.loc[:, 'ที่อยู่'].fillna("")
address = pd.Series(address ,name='status')

Provide a score to each user based on their providing address information or not

In [29]:
address_score = []
for x in address:
    if x == "":
        address_score.append(3)
    else:
        address_score.append(4)
address_score

[3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 3,
 4,
 3,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 3,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
